<a href="https://colab.research.google.com/github/mathengem/Algorithmic-Trading-Backtesting-in-python/blob/main/test2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install base58

In [4]:
!pip install bitcoin

In [5]:
!pip install bitstring

In [6]:
!pip install gmpy2


In [7]:
!pip install ecdsa

In [8]:
!pip install pycryptodome

In [9]:
!git clone https://github.com/iceland2k14/secp256k1.git

fatal: destination path 'secp256k1' already exists and is not an empty directory.


In [10]:
%cd secp256k1

/content/secp256k1


In [5]:
import secp256k1 as ice

#############################################################################
# Set the number of public keys to generate
#############################################################################
start_key = 1
num_public_keys = 2**32 #(about 4 billions of keys)
bits_to_store = 64
bytes_to_store = bits_to_store//8
rate_of_key_to_generate = 128
rate_of_key_to_store = rate_of_key_to_generate

interval_to_generate = range(start_key, start_key + num_public_keys, rate_of_key_to_store)
interval_to_store = range(start_key,start_key + num_public_keys,rate_of_key_to_store)

binary_mode = 1

#############################################################################


if (binary_mode == 1):

	f = open("public_keys_database.bin", "wb") #binary mode

	########################################generation#############################################
	public_keys=[]


	public_keys_complete=list(map(ice.scalar_multiplication,interval_to_generate)) #generates the public keys complete
	public_keys=list(map(lambda w: int(w[33-bytes_to_store:33].hex(),16),public_keys_complete)) #extract only the last bytes_to_store



	########################################writing the db##########################################
	for i in range(0,len(interval_to_store)):
		f.write(public_keys[i].to_bytes(bytes_to_store,byteorder='big'))  #writes each key

	f.close()

else:

	f = open("public_keys_database.txt", "w")

	#generation
	public_keys=[]

	for i in interval_to_generate:
		P4 = ice.scalar_multiplication(i)
		public_keys.append(P4[33-bytes_to_store:33].hex())

	#writing the db
	for i in range(0,len(interval_to_store)):
		f.write(public_keys[i])

	f.close()

In [14]:
import secp256k1 as ice

#############################################################################
# Set the number of public keys to generate
#############################################################################
start_key = 1
num_public_keys = 2**32 * 14 #(about 56 trillion keys)
bits_to_store = 64
bytes_to_store = bits_to_store//8
rate_of_key_to_generate = 128
rate_of_key_to_store = rate_of_key_to_generate

interval_to_generate = range(start_key, start_key + num_public_keys, rate_of_key_to_store)
interval_to_store = range(start_key, start_key + num_public_keys, rate_of_key_to_store)

binary_mode = 1

public_key = bytes.fromhex('03e1ea4a810944e6b5b1f598fae9984629c6cbcb9c0471f39c752809b44ed6ce11')

#############################################################################


if (binary_mode == 1):

	f = open("public_keys_database.bin", "wb") #binary mode

	########################################generation#############################################
	for i in range(0, len(interval_to_generate), rate_of_key_to_generate):
		public_keys=[]
		for j in range(i, min(i + rate_of_key_to_generate, len(interval_to_generate))):
			P4 = ice.scalar_multiplication(interval_to_generate[j])
			public_keys.append(int(P4[33-bytes_to_store:33].hex(), 16))

		########################################writing the db##########################################
		for k in range(len(public_keys)):
			f.write(public_keys[k].to_bytes(bytes_to_store, byteorder='big'))  #writes each key

		# Clear the RAM
		del public_keys
		public_keys = None

	f.close()

else:

	f = open("public_keys_database.txt", "w")

	#generation
	for i in range(0, len(interval_to_generate), rate_of_key_to_generate):
		public_keys=[]
		for j in range(i, min(i + rate_of_key_to_generate, len(interval_to_generate))):
			P4 = ice.scalar_multiplication(interval_to_generate[j])
			public_keys.append(P4[33-bytes_to_store:33].hex())

		#writing the db
		for k in range(len(public_keys)):
			f.write(public_keys[k] + '\n')

		# Clear the RAM
		del public_keys
		public_keys = None

	f.close()

In [6]:
import secp256k1 as ice
import random
import sys

#############################################################################
# Set the number of public keys to generate
#############################################################################

start_key = 1
num_public_keys = 2**32
bits_to_store = 64
bytes_to_store = bits_to_store//8
rate_of_key_to_generate = 128
rate_of_key_to_store = rate_of_key_to_generate

split_database = 16 #read only a fraction of the database to speedup the finding of the string

interval_to_generate = range(start_key,start_key + num_public_keys, rate_of_key_to_store)

interval_to_store = range(start_key,start_key + num_public_keys,rate_of_key_to_store)

binary_mode = 1



#########################################################################################

#pk = 0x3243 = 12867
#P = ice.scalar_multiplication(12867)
#P="0x6800b#b8a9dffe1709ceac95d7d06646188c2cb656c09cd2e717ec67487ce1be3"


#############generates random private key and public key#################################
pk= random.randint(start_key, start_key + num_public_keys)
P = ice.scalar_multiplication(pk)
print()
print("This is the public key: " + P[1:33].hex())
print("I need to find this private key: "+str(pk))


###################subtraction of : P - 1G,  P - 2G, ..., P - rate_of_key*G################
substract_pub = ice.scalar_multiplication(1)
complete_pub= ice.point_loop_subtraction(rate_of_key_to_generate, P, substract_pub)


partial_pub=[] #need only the last bytes_to_store
P2=int(P[33-bytes_to_store:33].hex(),16).to_bytes(bytes_to_store,byteorder='big')
partial_pub.append(P2)

for i in range(1,rate_of_key_to_store+1):
	partial_pub.append(int(complete_pub[(i-1)*65+33-bytes_to_store:(i-1)*65+33].hex(),16).to_bytes(bytes_to_store,byteorder='big'))




################search in database##########################################################
with open("public_keys_database.bin", 'r+b') as f:

	s = f.read()
	l=len(s)

	for k in range(0,l, l//split_database):
		j=1
		for i in partial_pub:
			n = s[k:k+l//split_database].find(i)
			if n > -1:
				print()
				print("Private key found!!!")
				private_key = (n+k)//bytes_to_store*rate_of_key_to_generate + j
				print(private_key)
				P3 = ice.scalar_multiplication(private_key)
				pub_key=P3[1:33].hex()
				print((pub_key)[:])
				f.close()
				sys.exit(0)
			j=j+1


print("string not found")


f.close()


This is the public key: 77c20d4b86e3838bd346f70b9f51efc663d41e1b1cadacf87b09f424679079d4
I need to find this private key: 2807233377

Private key found!!!
2807233377
77c20d4b86e3838bd346f70b9f51efc663d41e1b1cadacf87b09f424679079d4


SystemExit: 0

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [8]:
import secp256k1 as ice
import random
import sys

#############################################################################
# Set the number of public keys to generate
#############################################################################

start_key = 1
num_public_keys = 2**32
bits_to_store = 64
bytes_to_store = bits_to_store//8
rate_of_key_to_generate = 128
rate_of_key_to_store = rate_of_key_to_generate

split_database = 16 #read only a fraction of the database to speedup the finding of the string

interval_to_generate = range(start_key,start_key + num_public_keys, rate_of_key_to_store)

interval_to_store = range(start_key,start_key + num_public_keys,rate_of_key_to_store)

binary_mode = 1



#########################################################################################

#pk = 0x3243 = 12867
#P = ice.scalar_multiplication(12867)
#P="0x6800b#b8a9dffe1709ceac95d7d06646188c2cb656c09cd2e717ec67487ce1be3"


#############generates random private key and public key#################################
pk= random.randint(start_key, start_key + num_public_keys)
P = ice.scalar_multiplication(pk)
print()
print("This is the public key: " + P[1:33].hex())
print("I need to find this private key: "+str(pk))


###################subtraction of : P - 1G,  P - 2G, ..., P - rate_of_key*G################
substract_pub = ice.scalar_multiplication(1)
complete_pub= ice.point_loop_subtraction(rate_of_key_to_generate, P, substract_pub)


partial_pub=[] #need only the last bytes_to_store
P2=int(P[33-bytes_to_store:33].hex(),16).to_bytes(bytes_to_store,byteorder='big')
partial_pub.append(P2)

for i in range(1,rate_of_key_to_store+1):
	partial_pub.append(int(complete_pub[(i-1)*65+33-bytes_to_store:(i-1)*65+33].hex(),16).to_bytes(bytes_to_store,byteorder='big'))




################search in database##########################################################
with open("public_keys_database.bin", 'r+b') as f:

	s = f.read()
	l=len(s)

	for k in range(0,l, l//split_database):
		j=1
		for i in partial_pub:
			n = s[k:k+l//split_database].find(i)
			if n > -1:
				print()
				print("Private key found!!!")
				private_key = (n+k)//bytes_to_store*rate_of_key_to_generate + j
				print(private_key)
				P3 = ice.scalar_multiplication(private_key)
				pub_key=P3[1:33].hex()
				print((pub_key)[:])
				# f.close() # This line is no longer needed as the 'with' statement handles file closing
				# sys.exit(0) # Commenting out this line to prevent premature exit
				break # Exit the inner loop after finding the key
			j=j+1


print("string not found")


# f.close() # This line is no longer needed as the 'with' statement handles file closing


This is the public key: b55275906987717ef498f564f89fd14d9520a09894bce36a07062656a0231b43
I need to find this private key: 2754393154

Private key found!!!
2754393154
b55275906987717ef498f564f89fd14d9520a09894bce36a07062656a0231b43
string not found


In [ ]:
from sympy import mod_inverse
import secp256k1 as ice
pub=ice.pub2upub('03e1ea4a810944e6b5b1f598fae9984629c6cbcb9c0471f39c752809b44ed6ce11')
N=0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

k=mod_inverse(2,N)
neg1=ice.point_negation(ice.scalar_multiplication(1))


def ters(Qx,Scalar):
     ScalarBin = str(bin(Scalar))[2:]
     le=len(ScalarBin)
     for i in range (1,le+1):
        if ScalarBin[le-i] == "0":
            Qx=ice.point_multiplication(k,Qx)
        else:
            Qx=ice.point_addition(Qx,neg1)
            Qx=ice.point_multiplication(k,Qx)
     return ice.point_to_cpub(Qx)


for x in range(1,65536):
         print(ters(pub,x))


In [11]:
import ecdsa
import hashlib
import base58
from Crypto.Hash import RIPEMD
import concurrent.futures

def has_equal_bits(n):
    """Checks if an integer has an equal number of 0 and 1 bits in its binary representation."""
    binary = bin(n)[2:]
    return binary.count('0') == binary.count('1')

def generate_bitcoin_address(public_key):
  """Generates a Bitcoin address from a public key."""
  sha256_hash = hashlib.sha256(public_key).digest()
  ripemd160_hash = RIPEMD.new(sha256_hash).digest()
  extended_ripemd160_hash = b'\x00' + ripemd160_hash
  sha256_hash_2 = hashlib.sha256(extended_ripemd160_hash).digest()
  sha256_hash_3 = hashlib.sha256(sha256_hash_2).digest()
  checksum = sha256_hash_3[:4]
  address_bytes = extended_ripemd160_hash + checksum
  address = base58.b58encode(address_bytes).decode('utf-8')
  return address

def find_private_key_in_range(start, end, target_address):
    for i in range(start, end):
        if has_equal_bits(i):
            private_key = i.to_bytes(32, 'big')
            sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
            vk = sk.get_verifying_key()
            x = vk.pubkey.point.x()
            y = vk.pubkey.point.y()
            compressed_public_key = b'\x02' + x.to_bytes(32, 'big') if y % 2 == 0 else b'\x03' + x.to_bytes(32, 'big')
            address = generate_bitcoin_address(compressed_public_key)
            if address == target_address:
                return private_key
    return None

def find_private_key_parallel(start, end, target_address, num_workers=4):
    step = (end - start) // num_workers
    ranges = [(start + i * step, start + (i + 1) * step) for i in range(num_workers)]

    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(find_private_key_in_range, r[0], r[1], target_address) for r in ranges]
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            if result:
                return result
    return None

# Define range and target address
start = 2**19
end = 2**20
target_address = "1HsMJxNiV7TLxmoF6uJNkydxPFDog4NQum"

# Attempt to find the private key in parallel
private_key = find_private_key_parallel(start, end, target_address)

if private_key:
    print(f"Private key found: {private_key.hex()}")
else:
    print("Private key not found in the specified range.")

Private key found: 00000000000000000000000000000000000000000000000000000000000d2c55


In [ ]:
import multiprocessing
from itertools import islice

def count_bits_in_chunk(start, end):
    unequal_count = 0
    equal_count = 0
    diff_1_count = 0
    diff_2_count = 0
    diff_3_count = 0
    diff_4_count = 0
    diff_5_count = 0
    max_deviation = 0

    for i in range(start, end):
        ones = 0
        zeros = 0
        n = i
        while n:
            if n & 1:
                ones += 1
            else:
                zeros += 1
            n >>= 1

        if ones != zeros:
            unequal_count += 1
            deviation = abs(ones - zeros)
            max_deviation = max(max_deviation, deviation)

            if deviation == 1:
                diff_1_count += 1
            elif deviation == 2:
                diff_2_count += 1
            elif deviation == 3:
                diff_3_count += 1
            elif deviation == 4:
                diff_4_count += 1
            elif deviation == 5:
                diff_5_count += 1
        else:
            equal_count += 1

    return unequal_count, equal_count, diff_1_count, diff_2_count, diff_3_count, diff_4_count, diff_5_count, max_deviation

def combine_results(results):
    combined = [sum(x) for x in zip(*results)]
    combined[-1] = max(x[-1] for x in results)  # max_deviation
    return combined

def count_bits_parallel(start, end, num_processes=None):
    if num_processes is None:
        num_processes = multiprocessing.cpu_count()

    chunk_size = (end - start) // num_processes
    ranges = [(start + i * chunk_size, start + (i + 1) * chunk_size) for i in range(num_processes)]
    ranges[-1] = (ranges[-1][0], end)  # Adjust last chunk to include the end

    with multiprocessing.Pool(num_processes) as pool:
        results = pool.starmap(count_bits_in_chunk, ranges)

    return combine_results(results)

# Define range of numbers to check
start = 2**19
end = 2**20  # Extended the range for large bit search

# Run the function and print results
unequal_count, equal_count, diff_1_count, diff_2_count, diff_3_count, diff_4_count, diff_5_count, max_deviation = count_bits_parallel(start, end)
print(f"Number of numbers with unequal bits: {unequal_count}")
print(f"Number of numbers with equal bits: {equal_count}")
print(f"Number of numbers differing by 1 bit: {diff_1_count}")
print(f"Number of numbers differing by 2 bits: {diff_2_count}")
print(f"Number of numbers differing by 3 bits: {diff_3_count}")
print(f"Number of numbers differing by 4 bits: {diff_4_count}")
print(f"Number of numbers differing by 5 bits: {diff_5_count}")
print(f"Maximum deviation from 50/50: {max_deviation}")


Number of numbers with unequal bits: 431910
Number of numbers with equal bits: 92378
Number of numbers differing by 1 bit: 0
Number of numbers differing by 2 bits: 167960
Number of numbers differing by 3 bits: 0
Number of numbers differing by 4 bits: 125970
Number of numbers differing by 5 bits: 0
Maximum deviation from 50/50: 20


In [ ]:
!g++ -std=c++11 -O2 -o bit_count bit_count.cpp -pthread


In [ ]:
!./bit_count


In [ ]:
!pip install ecdsa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.3/149.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
from ecdsa import ellipticcurve, curves

# Define the secp256k1 curve
curve = curves.SECP256k1

# Your 10 points
points = [
    (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930),
    (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018),
    (21262057306151627953595685090280431278183829487175876377991189246716355947009, 41749993296225487051377864631615517161996906063147759678534462689479575333124),
    (104059883622109321374094289636044428849728529177856482232626205340719788190730, 112122903140080327253741791678230372394936108416576609264408917599318947489825),
    (95440839670107969455973995843666399663662641812074432045896568980475242364517, 67400892360194400039319989411395972789004161889863182881857158544061243615929),
    (86454928033100054822938644242727206101601557724291916072342392316125160730507, 41929975541376036990359335647717381527212342035893043668288666074313354583455),
    (23971227478092690611538379282579297124953800109492367109857348294863777014350, 42342609885299334444880650568116455571250837701978463648617679521175848103706),
    (59030624050581421406270513075794029219285200067000787157698366092408912155912, 8128656248049033031875480515138402531865803579197128279783442554673902546095),
    (31809325515952716603993860438135646801954922215901515683284707473623428145741, 24377531977987817432088011602449325046972761657480460991275213238540072159220),
    (16339661702852967382840638396154683021742565846854739320600712521008743256863, 36728284022334234592863792440353097018527397507662959793213172092233334129261),
]

def is_tangent(point):
    """Checks if a point is a tangent point on secp256k1."""
    x, y = point
    P = ellipticcurve.Point(curve.curve, x, y)

    # Handle division by zero in slope calculation
    if y == 0:
        return False  # Not a tangent if y is zero

    # Calculate the tangent line slope
    m = (3 * P.x()**2 + curve.curve.a()) / (2 * P.y())

    # Check if the tangent line intersects the curve at only one point
    try:
        tangent_point = P + P
        if tangent_point == P:
            return True  # Tangent point
        else:
            return False  # Not a tangent point
    except Exception as e:
        # Handle potential exceptions (e.g., division by zero)
        print(f"Error: {e}")
        return False

# Iterate through the points and check for tangency
for point in points:
    if is_tangent(point):
        print(f"Point {point} is a tangent point.")
    else:
        print(f"Point {point} is not a tangent point.")


Point (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930) is not a tangent point.
Point (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018) is not a tangent point.
Point (21262057306151627953595685090280431278183829487175876377991189246716355947009, 41749993296225487051377864631615517161996906063147759678534462689479575333124) is not a tangent point.
Point (104059883622109321374094289636044428849728529177856482232626205340719788190730, 112122903140080327253741791678230372394936108416576609264408917599318947489825) is not a tangent point.
Point (95440839670107969455973995843666399663662641812074432045896568980475242364517, 67400892360194400039319989411395972789004161889863182881857158544061243615929) is not a tangent point.
Point (864549280331000548229386442427272061016015577

In [ ]:
from ecdsa import ellipticcurve, curves

# Define the secp256k1 curve
curve = curves.SECP256k1

# Points to check
points = [
    (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930),
    (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018),
    (21262057306151627953595685090280431278183829487175876377991189246716355947009, 41749993296225487051377864631615517161996906063147759678534462689479575333124),
    (104059883622109321374094289636044428849728529177856482232626205340719788190730, 112122903140080327253741791678230372394936108416576609264408917599318947489825),
    (95440839670107969455973995843666399663662641812074432045896568980475242364517, 67400892360194400039319989411395972789004161889863182881857158544061243615929),
    (86454928033100054822938644242727206101601557724291916072342392316125160730507, 41929975541376036990359335647717381527212342035893043668288666074313354583455),
    (23971227478092690611538379282579297124953800109492367109857348294863777014350, 42342609885299334444880650568116455571250837701978463648617679521175848103706),
    (59030624050581421406270513075794029219285200067000787157698366092408912155912, 8128656248049033031875480515138402531865803579197128279783442554673902546095),
    (31809325515952716603993860438135646801954922215901515683284707473623428145741, 24377531977987817432088011602449325046972761657480460991275213238540072159220),
    (16339661702852967382840638396154683021742565846854739320600712521008743256863, 36728284022334234592863792440353097018527397507662959793213172092233334129261),
    (102184347352432016820033851144057611798380677393006808006149235955473719938577, 59133195063798511001762835616747095376330916791910217089881680396376786483431),
    (55066263022277343669578718895168534326250603453777594175500187360389116729240, 32670510020758816978083085130507043184471273380659243275938904335757337482424),
]

def is_tangent(point):
    """Checks if a point is a tangent point on secp256k1."""
    x, y = point
    P = ellipticcurve.Point(curve.curve, x, y)

    # Handle division by zero in slope calculation
    if y == 0:
        return False  # Not a tangent if y is zero

    # Calculate the tangent line slope
    m = (3 * P.x()**2 + curve.curve.a()) / (2 * P.y())

    # Check if the tangent line intersects the curve at only one point
    try:
        tangent_point = P + P
        if tangent_point == P:
            return True  # Tangent point
        else:
            return False  # Not a tangent point
    except Exception as e:
        # Handle potential exceptions (e.g., division by zero)
        print(f"Error: {e}")
        return False

# Iterate through the points and check for tangency
for point in points:
    if is_tangent(point):
        print(f"Point {point} is a tangent point.")
    else:
        print(f"Point {point} is not a tangent point.")


Point (89565891926547004231252920425935692360644145829622209833684329913297188986597, 12158399299693830322967808612713398636155367887041628176798871954788371653930) is not a tangent point.
Point (103388573995635080359749164254216598308788835304023601477803095234286494993683, 37057141145242123013015316630864329550140216928701153669873286428255828810018) is not a tangent point.
Point (21262057306151627953595685090280431278183829487175876377991189246716355947009, 41749993296225487051377864631615517161996906063147759678534462689479575333124) is not a tangent point.
Point (104059883622109321374094289636044428849728529177856482232626205340719788190730, 112122903140080327253741791678230372394936108416576609264408917599318947489825) is not a tangent point.
Point (95440839670107969455973995843666399663662641812074432045896568980475242364517, 67400892360194400039319989411395972789004161889863182881857158544061243615929) is not a tangent point.
Point (864549280331000548229386442427272061016015577

In [ ]:
!!sudo apt-get install  sagemath


['',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 0%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 3%',
 '',
 'Reading package lists... 4%',
 '',
 'Reading package lists... 4%',
 '',
 'Reading package lists... 38%',
 '',
 'Reading package lists... 41%',
 '',
 'Reading package lists... 41%',
 '',
 'Reading package lists... 41%',
 '',
 'Reading package lists... 41%',
 '',
 'Reading package lists... 48%',
 '',
 'Reading package lists... 48%',
 '',
 'Reading package lists... 56%',
 '',
 'Reading package lists... 56%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 60%',
 '',
 'Reading package lists... 66%',
 '',
 'Reading package lists... 66%',
 '',
 'Reading pack

In [ ]:
import sage.all as sage

# Define the prime field for secp256k1
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

# Define the finite field GF(p)
F = sage.FiniteField(p)

# Define the secp256k1 curve: y^2 = x^3 + 7 over GF(p)
E = sage.EllipticCurve(F, [0, 7])

# Display the curve to verify
print("Elliptic curve secp256k1:", E)

# Define the polynomial x^3 + 7 over GF(p)
x = sage.PolynomialRing(F, 'x').gen()
poly = x**3 + 7

# Find all roots of the polynomial in GF(p)
roots = poly.roots(multiplicities=False)

# Display the roots
print("Roots of the polynomial x^3 + 7 ≡ 0 mod p (potential x-coordinates for points with y=0):")
print(roots)

# Check if any roots were found
if not roots:
    print("No points with y = 0 exist on the secp256k1 curve over GF(p).")
else:
    print("Points with y = 0 on secp256k1 curve:")
    for x_val in roots:
        print(f"({x_val}, 0)")

Elliptic curve secp256k1: Elliptic Curve defined by y^2 = x^3 + 7 over Finite Field of size 115792089237316195423570985008687907853269984665640564039457584007908834671663
Roots of the polynomial x^3 + 7 ≡ 0 mod p (potential x-coordinates for points with y=0):
[]
No points with y = 0 exist on the secp256k1 curve over GF(p).


In [ ]:
import sage.all as sage

# Define the prime field for secp256k1
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

# Define the finite field GF(p)
F = sage.FiniteField(p)

# Define the secp256k1 curve: y^2 = x^3 + 7 over GF(p)
E = sage.EllipticCurve(F, [0, 7])

# Define the order of the subgroup
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Define the subgroup
G = E.abelian_group()

# Get the generator of the subgroup
g = G.gen(0)

# Print the order of the subgroup
print("Order of the subgroup:", g.order())

Order of the subgroup: 115792089237316195423570985008687907852837564279074904382605163141518161494337


In [ ]:
import sage.all as sage

# Define the prime field for secp256k1
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

# Define the finite field GF(p)
F = sage.FiniteField(p)

# Define the secp256k1 curve: y^2 = x^3 + 7 over GF(p)
E = sage.EllipticCurve(F, [0, 7])

# Define the order of the subgroup
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141

# Get the generator of the subgroup
g = E.gen(0) # Get the generator directly from the curve

# Find the tangent points on the curve
tangent_points = []
for i in range(n):
    point = g * i
    if point.is_zero(): # Check if the point is the point at infinity
        tangent_points.append(point)

# Print the tangent points
print("Tangent points on the curve:", tangent_points)

In [ ]:
import sage.all as sage
from sage.all import EllipticCurve, GF # Import EllipticCurve and GF

# Define the secp256k1 parameters
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F # Corrected prime number
a = 0
b = 7

# G is defined as an EllipticCurvePoint object
G = EllipticCurve(GF(p), [a, b])(0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
                 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
n = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

# Define the points P and Q as EllipticCurvePoint objects
P = EllipticCurve(GF(p), [a, b])(16339661702852967382840638396154683021742565846854739320600712521008743256863,
                 36728284022334234592863792440353097018527397507662959793213172092233334129261)

Q = EllipticCurve(GF(p), [a, b])(63004655751848499058589887215149057167805706207141784701705408548476267919372,
                 33776887358425213876727286236887696644549240079971817446727643228044518554934)

# Function to perform the BSGS attack
def bsgs(P, Q, G, n, t):
    # Precomputation
    table = {}
    for m in range(1, 2^t + 1):
        table[m * G] = m  # Multiplication is now defined for EllipticCurvePoint

    # Find match
    for m in range(1, 2^t + 1):
        R_prime = Q - m * G  # Subtraction is now defined for EllipticCurvePoint
        if R_prime in table:
            # Return the sum of indices found
            return m + table[R_prime]

    return None

# Set the parameter t
t = 2048
result = bsgs(P, Q, G, n, t)
print(result)

None


In [ ]:
import sage.all as sage

# Define the prime field for secp256k1
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

# Define the finite field GF(p)
F = sage.FiniteField(p)

# Define the secp256k1 curve: y^2 = x^3 + 7 over GF(p)
E = sage.EllipticCurve(F, [0, 7])

# Define the points P and Q
Px = 16339661702852967382840638396154683021742565846854739320600712521008743256863
Py = 36728284022334234592863792440353097018527397507662959793213172092233334129261
P = E(Px, Py)

Qx = 63004655751848499058589887215149057167805706207141784701705408548476267919372
Qy = 33776887358425213876727286236887696644549240079971817446727643228044518554934
Q = E(Qx, Qy)

# Choose the parameter t
t = 20

# Precompute the table of values
table = {}
for m in range(2**t):
    table[m*E.gens()[0]] = m

# Calculate R = P + Q
R = P + Q

# Find a match in the table
for m in range(2**t):
    R_prime = R - m*E.gens()[0]
    if R_prime in table:
        n = table[R_prime]
        break

# Solve for k
k = m + n
print("Recovered private key k:", k)

In [ ]:
import sage.all as sage

# Define the prime field for secp256k1
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F

# Define the finite field GF(p)
F = sage.FiniteField(p)

# Define the secp256k1 curve: y^2 = x^3 + 7 over GF(p)
E = sage.EllipticCurve(F, [0, 7])

# Define the generator point G
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
G = E(Gx, Gy)

# Define the points P and Q
Px = 16339661702852967382840638396154683021742565846854739320600712521008743256863
Py = 36728284022334234592863792440353097018527397507662959793213172092233334129261
P = E(Px, Py)

Qx = 63004655751848499058589887215149057167805706207141784701705408548476267919372
Qy = 33776887358425213876727286236887696644549240079971817446727643228044518554934
Q = E(Qx, Qy)

# Compute the tangent slope λ_P at point P
# Ensure all calculations and intermediate values are within the finite field F
lambda_P = F((3*Px^2)) / F((2*Py))  # Perform calculations in F

# Compute the secant slope λ_QP through points P and Q
lambda_QP = F(Qy - Py) / F(Qx - Px) # Perform calculations in F

print("Tangent slope λ_P:", lambda_P)
print("Secant slope λ_QP:", lambda_QP)

# Recover k1 and k. Calculations now happen within the finite field.
k1 = (lambda_QP - lambda_P) / (F(Px) - F(Gx))
k = (lambda_QP - lambda_P) / (F(Qx) - F(Gx))

# Convert k1 and k to integers using the lift() method
k1 = k1.lift() # Use lift() to convert to a regular integer
k = k.lift()   # Use lift() to convert to a regular integer

print("Recovered private key k₁:", k1)
print("Recovered private key k:", k)

Tangent slope λ_P: 61301323990699329266042017764052613939818165536762363998680553784412397691649
Secant slope λ_QP: 74251555817464640277801685005385861617742941841022323908591482994720359786909
Recovered private key k₁: 53537231712578432712823405411101860250049293220899126008948497458324234869151
Recovered private key k: 7565946342682645743208181718677733004923230108129102802756822574162365590547


In [ ]:
# Define the SECP256K1 curve
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
E = EllipticCurve(GF(p), [0, 0, 0, 0, 7])

# Define the generator point G = (x, y)
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
G = E(Gx, Gy)

# Define the test point P = (x, y)
Px = 16339661702852967382840638396154683021742565846854739320600712521008743256863
Py = 36728284022334234592863792440353097018527397507662959793213172092233334129261
P = E(Px, Py)

# Compute the order of the curve
n = 115792089210356248762697446949407573530086143415290314195533631308867097853951

# Try a brute-force attack
for k in range(n):
    Q = k * G
    if Q == P:
        print("Found k:", k)
        break

Found k: 1024


In [ ]:
from sage.all import EllipticCurve, GF

# Define the SECP256K1 curve
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
E = EllipticCurve(GF(p), [0, 7])

# Define the generator point G = (x, y)
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
G = E(Gx, Gy)

# Define the point P
Px = 16339661702852967382840638396154683021742565846854739320600712521008743256863
Py = 36728284022334234592863792440353097018527397507662959793213172092233334129261
P = E(Px, Py)

# Define the point Q (P = k' * G and Q = 1024 * P)
Qx = 63004655751848499058589887215149057167805706207141784701705408548476267919372
Qy = 33776887358425213876727286236887696644549240079971817446727643228044518554934
Q = E(Qx, Qy)

# Compute k' such that P = k' * G
def find_scalar_G_to_P(P, G, Q, scalar):
    """
    Find k' such that P = k' * G, given that Q = scalar * P.

    Args:
        P: The target elliptic curve point.
        G: Generator point of the elliptic curve.
        Q: The point obtained by multiplying P with scalar.
        scalar: The scalar value used for Q = scalar * P.

    Returns:
        The scalar k' such that P = k' * G.
    """
    try:
        # Compute the scalar k' such that Q = scalar * P
        k_prime = discrete_log(Q, G, operation='+') // scalar
        return k_prime
    except ValueError as e:
        print("Failed to solve the discrete log:", e)
        return None

# Calculate the scalar k' for G to P
scalar = 1024
k_prime = find_scalar_G_to_P(P, G, Q, scalar)

# Print the result
if k_prime is not None:
    print(f"The scalar k' such that P = k' * G is: {k_prime}")
else:
    print("Failed to find k'.")


The scalar k' such that P = k' * G is: 1024


In [ ]:
from sage.all import EllipticCurve, GF, discrete_log

# Define the SECP256K1 curve
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
E = EllipticCurve(GF(p), [0, 7])

# Define the generator point G = (x, y)
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
G = E(Gx, Gy)

# Define the point P
Px = 102184347352432016820033851144057611798380677393006808006149235955473719938577
Py = 59133195063798511001762835616747095376330916791910217089881680396376786483431
P = E(Px, Py)

# Define the point Q (P = k' * G and Q = 1024 * P)
Qx = 90830050882153464509167337448657263960001043321462090241596750046042988205087
Qy = 14079190045718298233572168364911200660550524081186606246657004668663157479269
Q = E(Qx, Qy)

# Compute k' such that P = k' * G
def find_scalar_G_to_P(P, G, Q, scalar):
    """
    Find k' such that P = k' * G, given that Q = scalar * P.

    Args:
        P: The target elliptic curve point.
        G: Generator point of the elliptic curve.
        Q: The point obtained by multiplying P with scalar.
        scalar: The scalar value used for Q = scalar * P.

    Returns:
        The scalar k' such that P = k' * G.
    """
    try:
        # Debug: Output the input points and scalar
        print(f"Computing discrete log with points P: {P}, G: {G}, Q: {Q}, scalar: {scalar}")

        # Compute the scalar k' such that Q = scalar * P
        k_prime = discrete_log(Q, G, operation='+') // scalar

        # Debug: Output the result
        print(f"Computed k' = {k_prime}")

        return k_prime
    except ValueError as e:
        print("Failed to solve the discrete log:", e)
        return None

# Calculate the scalar k' for G to P
scalar = 1024
k_prime = find_scalar_G_to_P(P, G, Q, scalar)

# Print the result
if k_prime is not None:
    print(f"The scalar k' such that P = k' * G is: {k_prime}")
else:
    print("Failed to find k'.")


Computing discrete log with points P: (102184347352432016820033851144057611798380677393006808006149235955473719938577 : 59133195063798511001762835616747095376330916791910217089881680396376786483431 : 1), G: (55066263022277343669578718895168534326250603453777594175500187360389116729240 : 32670510020758816978083085130507043184471273380659243275938904335757337482424 : 1), Q: (90830050882153464509167337448657263960001043321462090241596750046042988205087 : 14079190045718298233572168364911200660550524081186606246657004668663157479269 : 1), scalar: 1024


In [ ]:
import gmpy2
import gc
from sage.all import EllipticCurve, GF

# Define the SECP256K1 curve
p = gmpy2.mpz("115792089237316195423570985008687907853269984665640564039457584007908834671663")
E = EllipticCurve(GF(p), [0, 7])

# Define the generator point G = (x, y)
Gx = gmpy2.mpz("55066263022277343669578718895168534326250603453777594175500187360389116729240")
Gy = gmpy2.mpz("32670510020758816978083085130507043184471273380659243275938904335757337482424")
G = E(Gx, Gy)

# Define the point P
Px = gmpy2.mpz("102184347352432016820033851144057611798380677393006808006149235955473719938577")
Py = gmpy2.mpz("59133195063798511001762835616747095376330916791910217089881680396376786483431")
P = E(Px, Py)

# Define the point Q (P = k' * G and Q = 1024 * P)
Qx = gmpy2.mpz("90830050882153464509167337448657263960001043321462090241596750046042988205087")
Qy = gmpy2.mpz("14079190045718298233572168364911200660550524081186606246657004668663157479269")
Q = E(Qx, Qy)

# Divide-and-Conquer function to find the scalar in batches with garbage collection
def find_scalar_in_batches(P, G, Q, scalar, batch_size):
    """
    Find k' such that P = k' * G in batches, given that Q = scalar * P.

    Args:
        P: The target elliptic curve point.
        G: Generator point of the elliptic curve.
        Q: The point obtained by multiplying P with scalar.
        scalar: The scalar value used for Q = scalar * P.
        batch_size: Size of the batch to search in each iteration.

    Returns:
        The scalar k' such that P = k' * G.
    """
    k_prime = gmpy2.mpz(0)
    current_batch_start = gmpy2.mpz(0)

    # Iterate in batches until the scalar is found
    while True:
        current_batch_end = current_batch_start + batch_size

        print(f"Searching batch: {current_batch_start} to {current_batch_end}")

        try:
            # Perform garbage collection at the start of each batch
            gc.collect()

            # Compute the discrete log for the current batch
            for k in range(current_batch_start, current_batch_end):
                k_G = k * G  # Compute k * G

                # Check if we've found the match: P = k * G
                if k_G == P:
                    k_prime = k
                    return k_prime

        except ValueError as e:
            print(f"Failed batch {current_batch_start} to {current_batch_end}: {e}")

        # Move to the next batch
        current_batch_start = current_batch_end

# Parameters
batch_size = gmpy2.mpz(1000000)  # Adjusted batch size to avoid memory overload

# Enable automatic garbage collection
gc.enable()

# Search for k' using divide-and-conquer with garbage collection
scalar = gmpy2.mpz(1024)
k_prime = find_scalar_in_batches(P, G, Q, scalar, batch_size)

# Print the result
if k_prime is not None:
    print(f"The scalar k' such that P = k' * G is: {k_prime}")
else:
    print("Failed to find k'.")


Searching batch: 0 to 1000000
Searching batch: 1000000 to 2000000
Searching batch: 2000000 to 3000000
Searching batch: 3000000 to 4000000
Searching batch: 4000000 to 5000000
Searching batch: 5000000 to 6000000
Searching batch: 6000000 to 7000000
Searching batch: 7000000 to 8000000
Searching batch: 8000000 to 9000000
Searching batch: 9000000 to 10000000
Searching batch: 10000000 to 11000000
Searching batch: 11000000 to 12000000
Searching batch: 12000000 to 13000000


In [12]:
import secp256k1 as ice
from bitstring import BitArray
import bitcoin


print("Making Binary Data-Base")


target_public_key = "030d282cf2ff536d2c42f105d0b8588821a915dc3f9a05bd98bb23af67a2e92a5b"

target = ice.pub2upub(target_public_key)


num = 10000 # number of keys.

sustract= 1 #amount to subtract each time, If you modify this, use the same amount to scan.

Low_m= 10

lm= num // Low_m
print(lm)

sustract_pub= ice.scalar_multiplication(sustract)

res= ice.point_loop_subtraction(lm, target, sustract_pub)
binary = ''
for t in range (lm):

    h= (res[t*65:t*65+65]).hex()
    hc= int(h[2:], 16)


    if str(hc).endswith(('0','2','4','6','8')):
        A="0"
        binary+= ''.join(str(A))

    if str(hc).endswith(('1','3','5','7','9')):
        A="1"
        binary+= ''.join(str(A))


my_str = bytes(BitArray(bin=binary))

binary_file = open('data-base.bin', 'ab')
binary_file.write(my_str)
binary_file.close()

for i in range (1,Low_m):
    lm_upub= sustract_pub= ice.scalar_multiplication((lm*i)*sustract)

    A1= ice.point_subtraction(target, lm_upub)

    sustract_pub= ice.scalar_multiplication(sustract)

    res= ice.point_loop_subtraction(lm, A1, sustract_pub)

    binary = ''
    for t in range (lm):

        h= (res[t*65:t*65+65]).hex()
        hc= int(h[2:], 16)


        if str(hc).endswith(('0','2','4','6','8')):
            A="0"
            binary+= ''.join(str(A))

        if str(hc).endswith(('1','3','5','7','9')):
            A="1"
            binary+= ''.join(str(A))


    my_str = bytes(BitArray(bin=binary))

    binary_file = open('data-base.bin', 'ab')
    binary_file.write(my_str)
    binary_file.close()

Making Binary Data-Base
1000


In [13]:
import secp256k1 as ice
from bitstring import BitArray

def divide_integer_into_parts(n, num_parts):
    # Convert the integer to a binary string
    bin_str = bin(n)[2:]

    # Calculate the number of bits in each part
    bits_per_part = len(bin_str) // num_parts

    # Handle the case when bits_per_part is 0
    if bits_per_part == 0:
        return [bin_str]

    # Split the binary string into parts of equal length
    parts = [bin_str[i:i + bits_per_part] for i in range(0, len(bin_str), bits_per_part)]

    return parts

def write_to_binary_file(parts, file_name='result256.bin'):
    binary_file = open(file_name, 'ab')

    for part in parts:
        # Convert binary string to bytes
        byte_value = int(part, 2).to_bytes((len(part) + 7) // 8, byteorder='big')  # +7 ensures rounding up to the nearest byte
        binary_value = bytes(BitArray(bin=part))  # Create bytes from the binary string
        binary_file.write(binary_value)

    binary_file.close()

# Example integer
n = 10607741884884178603537133864630296054456886886068096376455927186044441555811

# Number of parts to divide into
num_parts = 256

# Divide the integer into parts
parts = divide_integer_into_parts(n, num_parts)

# Write those parts to a binary file
write_to_binary_file(parts)

print(f"{num_parts} parts have been written to 'result256.bin'.")

256 parts have been written to 'result256.bin'.


In [ ]:
import secp256k1 as ice
import random
from bitstring import BitArray



print("Scanning Binary Sequence")

#Pk: 1033162084
#cPub: 03e1ea4a810944e6b5b1f598fae9984629c6cbcb9c0471f39c752809b44ed6ce11

#range
start= 10607741884884178603537133864630296054456886886068096376455927100000000000000
end=   10607741884884178603537133864630296054456886886068096376455927900000000000000


while True:

    pk= random.randint(start, end)

    target = ice.scalar_multiplication(pk)

    num = 64 # collision margin.

    sustract= 1 # #amount to subtract each time.

    sustract_pub= ice.scalar_multiplication(sustract)

    res= ice.point_loop_subtraction(num, target, sustract_pub)

    binary = ''

    for t in range (num):

        h= (res[t*65:t*65+65]).hex()
        hc= int(h[2:], 16)


        if str(hc).endswith(('0','2','4','6','8')):
            A="0"
            binary+= ''.join(str(A))

        if str(hc).endswith(('1','3','5','7','9')):
            A="1"
            binary+= ''.join(str(A))


    my_str = binary

    b = bytes(BitArray(bin=my_str))


    file = open("result256.bin", "rb")

    dat = bytes(file.read())

    if b  in dat:
        s = b
        f = dat
        inx = f.find(s)*sustract
        inx_0=inx
        Pk = (int(pk) + int(inx_0))+int(inx_0)*7

        data = open("win.txt","a")
        data.write("Pk:"+" "+str(Pk)+"\n")
        data.close()
        break

In [9]:
import secp256k1 as ice
from bitstring import BitArray
import bitcoin


print("Making Binary Data-Base")


target_public_key = "0230210c23b1a047bc9bdbb13448e67deddc108946de6de639bcc75d47c0216b1b"

target = ice.pub2upub(target_public_key)


num = 2000000000 # number of keys.

sustract = 1 #amount to subtract each time.

Low_m= 200

#2^28 / 2^ 26 = 4
#2^29 / 2^26 = 8
#2^30 / 2^26 = 16
#2^32 / 2^26 = 64

lm= num // Low_m


sustract_pub= ice.scalar_multiplication(sustract)

res= ice.point_loop_subtraction(lm, target, sustract_pub)
binary = ''
for t in range (lm):

    h= (res[t*65:t*65+65]).hex()
    hc= int(h[2:], 16)


    if str(hc).endswith(('0','2','4','6','8')):
        A="0"
        binary+= ''.join(str(A))

    if str(hc).endswith(('1','3','5','7','9')):
        A="1"
        binary+= ''.join(str(A))


my_str = bytes(BitArray(bin=binary))

binary_file = open('130Bit.bin', 'wb')
binary_file.write(my_str)
binary_file.close()

for i in range (1,Low_m):

    mem= ice.to_cpub(ice.scalar_multiplication(lm).hex())

    Apk= bitcoin.multiply(mem, i)

    Apk_upu= ice.pub2upub(Apk)

    A1= ice.point_addition(target, Apk_upu)

    sustract_pub= ice.scalar_multiplication(sustract)

    res= ice.point_loop_subtraction(lm, A1, sustract_pub)

    binary = ''
    for t in range (lm):

        h= (res[t*65:t*65+65]).hex()
        hc= int(h[2:], 16)


        if str(hc).endswith(('0','2','4','6','8')):
            A="0"
            binary+= ''.join(str(A))

        if str(hc).endswith(('1','3','5','7','9')):
            A="1"
            binary+= ''.join(str(A))


    my_str = bytes(BitArray(bin=binary))

    binary_file = open('65Bit.bin', 'ab')
    binary_file.write(my_str)
    binary_file.close()


Making Binary Data-Base


KeyboardInterrupt: 

In [12]:
def divide_integer_into_parts(n, num_parts):
    # Convert the integer to a binary string
    bin_str = bin(n)[2:]

    # Calculate the number of bits in each part
    bits_per_part = len(bin_str) // num_parts

    # Handle the case when bits_per_part is 0
    if bits_per_part == 0:
        return [bin_str]

    # Split the binary string into parts of equal length
    parts = [bin_str[i:i + bits_per_part] for i in range(0, len(bin_str), bits_per_part)]

    return parts

# Given integer
n = 10607741884884178603537133864630296054456886886068096376455927186044441555811

# Divide into 256 parts
num_parts = 256
parts = divide_integer_into_parts(n, num_parts)

# Write parts to a binary file
with open('result256.bin', 'wb') as f:
    for part in parts:
        # Convert binary string to bytes (padded to ensure 1 byte is the minimum)
        byte_value = int(part, 2).to_bytes((len(part) + 7) // 8, byteorder='big')  # +7 ensures rounding up to the nearest byte
        f.write(byte_value)

print(f"{num_parts} parts have been written to 'result256.bin'.")

256 parts have been written to 'result256.bin'.


In [15]:
import secp256k1 as ice

def _point_subtraction(pubkey1_bytes, pubkey2_bytes):
    return ice.point_subtraction(pubkey1_bytes, pubkey2_bytes)

def point_subtraction(pubkey1_bytes, pubkey2_bytes):
    res = _point_subtraction(pubkey1_bytes, pubkey2_bytes)
    return bytes(bytearray(res))

target_public_key = "026ecabd2d22fdb737be21975ce9a694e108eb94f3649c586cc7461c8abf5da71a"
target = ice.pub2upub(target_public_key)

num = 2**23
subtract= 2**10
subtract_pub= ice.scalar_multiplication(subtract)

with open('data-base.bin', 'wb') as binary_file:
    current_pubkey = target
    byte_accumulator = 0
    bit_position = 0

    for _ in range(num):
        current_pubkey = point_subtraction(current_pubkey, subtract_pub)

        binary_data = int(current_pubkey.hex(), 16)

        bit = 0 if str(binary_data).endswith(('0', '2', '4', '6', '8')) else 1

        byte_accumulator = (byte_accumulator << 1) | bit
        bit_position += 1
        if bit_position == 8:
            binary_file.write(byte_accumulator.to_bytes(1, byteorder='big'))
            byte_accumulator = 0
            bit_position = 0

    if bit_position > 0:
        byte_accumulator <<= (8 - bit_position)
        binary_file.write(byte_accumulator.to_bytes(1, byteorder='big'))


In [11]:
import secp256k1 as ice
from bitstring import BitArray
import gc  # Import garbage collection module

#########################DATABASE PARAMETERS#########################
prefix = '255_32'
data_base_name = 'data-base' + prefix + '.bin'

num_public_keys = 2**254  # Number of keys in the range from 2^254 to 2^255
num_batches = 14  # Number of batches, each containing 2^32 keys
num_bytes_db = num_batches // 8  # 1 bit for each batch
sustract = 2**0  # Amount to subtract each time, If you modify this, use the same amount to scan.
sustract_pub = ice.scalar_multiplication(sustract)
space_covered = num_public_keys * sustract  # 2**254 = very large space covered

# Target public key
target_public_key = "03f760d768600e639758836fda5da546c10b1d84886165a5a1a4586192518aaf77"
target = ice.pub2upub(target_public_key)

# Low memory optimization
Low_m = 2**8  # To save RAM
lm = num_public_keys // Low_m

print("Making Binary Data-Base")
print()

# Initialize the binary file
with open(data_base_name, 'wb') as binary_file:
    pass

for batch_index in range(num_batches):
    print(f"Processing batch {batch_index + 1} of {num_batches}")

    # Calculate the starting point for this batch
    batch_start = 2**254 + batch_index * 2**32
    batch_start_pub = ice.scalar_multiplication(batch_start * sustract)

    # Subtract the batch start point from the target
    A1 = ice.point_subtraction(target, batch_start_pub)

    # Process the batch in chunks
    for i in range(Low_m):
        lm_upub = ice.scalar_multiplication((lm * i) * sustract)
        A1_chunk = ice.point_subtraction(A1, lm_upub)

        # Instead of generating one large array, process in smaller chunks
        chunk_size = 1000000
        res = bytearray()
        for j in range(0, lm, chunk_size):
            temp_res = ice.point_loop_subtraction(chunk_size, A1_chunk, sustract_pub)
            res.extend(temp_res)
            A1_chunk = ice.point_subtraction(A1_chunk, ice.scalar_multiplication(chunk_size)) # Update A1_chunk for next iteration

            # Trigger garbage collection to free up memory
            gc.collect()

        my_str = BitArray(res[t * 65 + 64] % 2 for t in range(lm))  # 0 if even, 1 if odd

        with open(data_base_name, 'ab') as binary_file:
            my_str.tofile(binary_file)

        # Trigger garbage collection to free up memory
        gc.collect()

print("Keys generated: " + str(num_public_keys))
print("Space covered by the keys spread with distance " + str(sustract) + " in the database : " + str(space_covered))

AttributeError: module 'secp256k1' has no attribute 'scalar_multiplication'

In [2]:
!git clone https://github.com/bobmitch23/discrete-log.git

Cloning into 'discrete-log'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 110 (delta 25), reused 92 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (110/110), 22.20 KiB | 3.17 MiB/s, done.
Resolving deltas: 100% (25/25), done.


In [3]:
%cd discrete-log

/content/discrete-log


In [8]:
%cd python

/content/discrete-log/python


In [9]:
!ls

discrete_log  discrete_log_test.py
